# Welcome to 15rock!

## README

- This notebook is temporary! Please save your work locally if required.
- The user below is a demo user COMMONLY accessed by all users. This means your tables can be seen, modified and  laughed at. If you want your own user, register here: xyz.com

In [11]:
import fifteenrock

In [12]:
url = 'https://bddcxzm20m.execute-api.us-east-1.amazonaws.com/staging'
credentials =  {"type": "user_name",
                "user_name": 'demo-user',
                "password": "mydemopassword"}


In [13]:
dc = fifteenrock.get_database_client(url=url, credentials=credentials)

# Raw Data

In [14]:
result = dc.query_sql('SELECT * from demo.customer;')

In [15]:
result

[{'age': 35, 'id': 1, 'name': 'Joe'},
 {'age': 25, 'id': 2, 'name': 'Bob'},
 {'age': 22, 'id': 3, 'name': 'Gail'}]

In [16]:
create_query = '''
create table if not exists demo.scoring_model (    id int,
                                 customer_id int  REFERENCES demo.customer(id),
                                 score float,
                                 PRIMARY KEY(id));
'''

dc.execute_command(create_query)

{'result': 'SQL command executed successfully'}

# Risk Model: Analysis

In [7]:
def has_achieved_wisdom(age, threshold):
    return age > threshold

customer_data = dc.query_sql("select id, age from demo.customer;")


# customer_data  looks like   [{'id': 1, 'age': 35}, {'id': 2, 'age': 25}, {'id': 3, 'age': 24}]

# Complex Risk Model
scores = []
for score_id, customer in enumerate(customer_data, 1):
    age = customer['age']
    if has_achieved_wisdom(age, 30):
        risk_score = age * 0.2
    else:
        risk_score = age * 10
    print(score_id, customer['id'], risk_score)
    scores.append([score_id, customer['id'], risk_score])

# # The values: lower the better
# # Columns below: ScoreId, CustomerId, RiskScore
# # scores = [[1, 1, 7.0],
# #           [2, 2, 50],
# #           [3, 3, 48]]
#

dc.execute_command("TRUNCATE demo.scoring_model;")

dc.insert_sql(table='demo.scoring_model',
              columns=["id", "customer_id", "score"],
              values=scores)


1 1 7.0
2 2 250
3 3 220


# Reporting/Analysis

In [9]:
result = dc.query_sql(
        'SELECT customer.name, scoring_model.score from demo.customer as customer, demo.scoring_model as scoring_model'
        ' where customer.id = scoring_model.customer_id')

In [10]:
result

[{'name': 'Joe', 'score': 7.0},
 {'name': 'Bob', 'score': 250.0},
 {'name': 'Gail', 'score': 220.0}]